<a href="https://colab.research.google.com/github/vezvaeia/TREC-CAR/blob/main/BM25%2BBERT%2BT5%2BBART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Installing Libraries and Downloading Datasets

### Install Libraries

In [3]:
# !pip install pygaggle
!pip install torch==1.4.0.
!pip install remarshal
!pip install cbor
!pip install cbor2
!pip install trec-car-tools
!pip install transformers
!pip install sentencepiece
!pip install pyserini==0.12.0

### Import Libraries

In [4]:
from tqdm import tqdm
from google.colab import drive
from pathlib import Path
import trec_car
from trec_car import read_data
import json
import os
import cbor
import cbor2
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import random
import numpy as np
from transformers import pipeline
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
from pyserini.search import SimpleSearcher
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoT5
from pygaggle.rerank.transformer import MonoBERT

### Set environment

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Cuda available: ",torch.cuda.is_available())
if torch.cuda.is_available():
    print("Current device: ",  torch.cuda.current_device())

seed = 204920
seed2 = 293652

random.seed(seed2)
np.random.seed(seed2)
torch.manual_seed(seed2)

if device.type == 'cuda':
    torch.cuda.manual_seed_all(seed)

Cuda available:  True
Current device:  0


### Download, Move, and Extract datasets

In [6]:
from google.colab import drive
from pathlib import Path

%cd /content/drive/MyDrive/IR-project/

# !wget trec-car.cs.unh.edu/datareleases/v2.0/paragraphCorpus.v2.0.tar.xz -O corpus.tar.xz
# !wget trec-car.cs.unh.edu/datareleases/v2.3/paragraphCorpus-splits.tar.xz -O corpus-chunked.tar.xz
# !wget trec-car.cs.unh.edu/results-Y3/trec-car-y3-runs-and-eval.tar.gz -O runs.tar.gz
# !wget trec-car.cs.unh.edu/datareleases/v2.3/benchmarkY3test.public.tar.gz -O outlines.tar.gz

# !cp /content/drive/MyDrive/IR-project/corpus.tar.xz /content/corpus.tar.xz
# !cp /content/drive/MyDrive/IR-project/corpus-chunked.tar.xz /content/corpus-chunked.tar.xz
!cp /content/drive/MyDrive/IR-project/runs.tar.gz /content/runs.tar.gz
!cp /content/drive/MyDrive/IR-project/outlines.tar.gz /content/outlines.tar.gz
!cp /content/drive/MyDrive/IR-project/paragraph-dictionary.json /content/paragraph-dictionary.json
!cp /content/drive/MyDrive/IR-project/topic-queries.json /content/topic-queries.json
!cp /content/drive/MyDrive/IR-project/bm25-docs.json /content/bm25-docs.json

%cd /content/

# !tar -xf corpus.tar.xz
# !tar -xf corpus-chunked.tar.xz
!tar -xf runs.tar.gz
!tar -xf outlines.tar.gz

/content/drive/MyDrive/IR-project
/content


# 2.Data Preparation

### Extracting topics and outlines

In [7]:
# for page in read_data.iter_outlines(open('/content/benchmarkY3test.public/benchmarkY3test.cbor-outlines.cbor', 'rb')):
#     print(page.page_id, page.page_name, page.page_type)
#     print(page.get_text_with_headings())

test_topics = {}
with open('/content/benchmarkY3test.public/benchmarkY3test.cbor-outlines.cbor', 'rb') as test_topics_file:
    raw_data = cbor.load(test_topics_file)
    print(raw_data)
    raw_data = cbor.load(test_topics_file)
    print(len(raw_data))
    
    for index, raw_topic in enumerate(raw_data):
        topic_id = raw_topic[2].decode("utf-8")
        topic_name = raw_topic[1]
        test_topics[topic_id] = {'title': topic_name, 'facets':{}}
        
        for facet in raw_topic[3]:
            facet_id = facet[2].decode("utf-8")
            facet_name = facet[1]
            test_topics[topic_id]['facets'][facet_id] = facet_name

for topic in test_topics:
    print(topic, ' : ', test_topics[topic])
            

['CAR', [1], [0, [[0, 'tqa2', 'en-us', 'tqa', []]], 'benchmarkY3test', ['TREC-CAR Dataset by Laura Dietz, Ben Gamari is licensed under a Creative Commons Attribution-ShareAlike 3.0 Unported License. Based on a work at http://data.allenai.org/tqa/ .', 'Used as test data for year 3 of the TREC Complex Answer Retrieval track.'], []]]
132
tqa2:L_0002  :  {'title': 'Earth Science and its branches', 'facets': {'T_0016': 'Geology', 'T_0017': 'Oceanography', 'T_0018': 'Climatology and Meteorology', 'T_0019': 'Environmental Science', 'T_0020': 'Astronomy'}}
tqa2:L_0003  :  {'title': 'erosion and deposition by flowing water', 'facets': {'T_0021': 'how flowing water causes erosion and deposition', 'T_0022': 'water speed and erosion', 'T_0023': 'particle size and erosion', 'T_0025': 'surface water', 'T_0028': 'formation of waterfalls', 'T_0029': 'slow-flowing rivers', 'T_0031': 'flood waters', 'T_0032': 'groundwater, caves, sinkholes'}}
tqa2:L_0004  :  {'title': 'erosion and deposition by waves', 

### Extracting paragraphs and saving into corpus.jsonl

In [11]:
import trec_car
from trec_car import read_data
from tqdm import tqdm

with open('corpus.jsonl', 'w') as file:

    for i in tqdm(range(1, 31)):
        read_file_name = 'paragraphCorpus/paragraphCorpus-' + str(i) + '.cbor'
        for para in read_data.iter_paragraphs(open(read_file_name, 'rb')):
            json.dump({"id": para.para_id, "contents": para.get_text()}, file)
            file.write('\n')


100%|██████████| 30/30 [18:40<00:00, 37.34s/it]


In [19]:
!cp corpus.jsonl /content/corpus-folder/corpus.jsonl

### Creating index

In [20]:
!python -m pyserini.index -collection JsonCollection -generator DefaultLuceneDocumentGenerator -threads 1 -input corpus-folder -index index-folder -storePositions -storeDocvectors -storeRaw

2021-05-06 22:09:48,069 INFO  [main] index.IndexCollection (IndexCollection.java:631) - Setting log level to INFO
2021-05-06 22:09:48,071 INFO  [main] index.IndexCollection (IndexCollection.java:634) - Starting indexer...
2021-05-06 22:09:48,071 INFO  [main] index.IndexCollection (IndexCollection.java:635) - ============ Loading Parameters ============
2021-05-06 22:09:48,071 INFO  [main] index.IndexCollection (IndexCollection.java:636) - DocumentCollection path: corpus-folder
2021-05-06 22:09:48,071 INFO  [main] index.IndexCollection (IndexCollection.java:637) - CollectionClass: JsonCollection
2021-05-06 22:09:48,071 INFO  [main] index.IndexCollection (IndexCollection.java:638) - Generator: DefaultLuceneDocumentGenerator
2021-05-06 22:09:48,072 INFO  [main] index.IndexCollection (IndexCollection.java:639) - Threads: 1
2021-05-06 22:09:48,072 INFO  [main] index.IndexCollection (IndexCollection.java:640) - Stemmer: porter
2021-05-06 22:09:48,072 INFO  [main] index.IndexCollection (Index

### Searching and ranking by BM25

In [47]:
searcher = SimpleSearcher('index-folder')
searcher.set_bm25(0.9, 0.4)
searcher.set_rm3(10, 10, 0.5)

topic_docs = {}
topic_queries = {}

for topic in tqdm(test_topics):
    for facet in test_topics[topic]['facets']:
     
        full_topic_id = topic + '/' + facet
        search_query = test_topics[topic]['title'] + ' ' + test_topics[topic]['facets'][facet]
        topic_queries[full_topic_id] = search_query
        topic_docs[full_topic_id] = []

        hits = searcher.search(search_query, 1000)
        for i in range(len(hits)):
            topic_docs[full_topic_id].append(hits[i].docid)


100%|██████████| 132/132 [04:32<00:00,  2.06s/it]


### Gathering all paragraph-ids from all the runs and qrels 

In [48]:
import os

all_paragraph_ids = set([])

for topic in topic_docs:
    for docid in topic_docs[topic]:
        all_paragraph_ids.add(docid)
        
for filename in os.listdir('run'):
    # print(filename)
    with open(os.path.join('run', filename), 'r') as runfile:
       for line in runfile:
           elements = line.strip().split('\t')
           if len(elements) != 6:
               print('Fuck!')
           all_paragraph_ids.add(elements[2])

with open('trec-car-benchmarkY3test-section.qrels', 'r') as qrels:
    for line in qrels:
        elements = line.strip().split()
        if len(elements) != 4:
               print('Fuck!')
        all_paragraph_ids.add(elements[2])

print(len(all_paragraph_ids))

340347


### Saving all used paragraphs into paragraph-dictionary

In [50]:
paragraph_dictionary = {}
cnt = 0

for i in tqdm(range(1, 31)):

    corpus_chunk_file_name = 'paragraphCorpus/paragraphCorpus-' + str(i) + '.cbor'
    chunk_dictionary = {}
    
    for para in read_data.iter_paragraphs(open(corpus_chunk_file_name, 'rb')):
        chunk_dictionary[para.para_id] = para.get_text()

    for para_id in all_paragraph_ids:
        if para_id in chunk_dictionary:
            paragraph_dictionary[para_id] = chunk_dictionary[para_id]
            cnt += 1
        
            if cnt % 10000 == 0:
                print(cnt)


  3%|▎         | 1/30 [00:24<11:52, 24.58s/it]

10000


  7%|▋         | 2/30 [00:48<11:20, 24.30s/it]

20000


 10%|█         | 3/30 [01:11<10:43, 23.85s/it]

...

310000
320000


 97%|█████████▋| 29/30 [11:18<00:23, 23.22s/it]

330000


100%|██████████| 30/30 [11:36<00:00, 23.22s/it]

340000


### Saving paragraph-dictionary and BM25 results

In [52]:
import json

print(len(paragraph_dictionary.keys()))    

with open('paragraph-dictionary.json', 'w') as file:
    json.dump(paragraph_dictionary, file)

with open('bm25-docs.json', 'w') as file:
    json.dump(topic_docs, file)

with open('topic-queries.json', 'w') as file:
    json.dump(topic_queries, file)

with open('/content/drive/MyDrive/IR-project/topic-queries.json', 'w') as file:
    json.dump(topic_queries, file)

with open('/content/drive/MyDrive/IR-project/bm25-docs.json', 'w') as file:
    json.dump(topic_docs, file)

with open('/content/drive/MyDrive/IR-project/paragraph-dictionary.json', 'w') as file:
    json.dump(paragraph_dictionary, file)

340347


### Restoring paragraph-dictionary, topic_docs, and topic_queries

In [13]:
paragraph_dictionary = {}
with open('paragraph-dictionary.json', 'r') as file:
    paragraph_dictionary = json.load(file)

topic_docs = {}
with open('bm25-docs.json', 'r') as file:
    topic_docs = json.load(file)

topic_queries = {}
with open('topic-queries.json', 'r') as file:
    topic_queries = json.load(file)

for index, key in enumerate(paragraph_dictionary):
    print(key, ' : ', paragraph_dictionary[key])
    if index > 10:
        break

for index, key in enumerate(topic_docs):
    print(key, ' : ', topic_docs[key])
    if index > 10:
        break

for index, key in enumerate(topic_queries):
    print(key, ' : ', topic_queries[key])
    if index > 10:
        break

07bd68d0fbd0d57ae786c7cd2637d625a1663286  :  The mollusks escape into the irrigation canal system, attacking livestock, a lock keeper, a trysting couple and others. Navy divers locate the group of mollusks in the canal system, and use explosives to destroy them.
063c6f44526be4611d73c34c0ac4d4f1f6384172  :  The 110 MW Crescent Dunes Solar Energy Project is the world’s first utility-scale facility to use molten salt power tower energy storage. It has 10,347 tracking mirrors  (heliostats)  that follow the sun and reflect and concentrate sunlight onto a heat exchanger, a receiver, atop a 640 foot tower. Crescent Dunes has 10 hours of storage and will deliver 500,000 MW hours of electricity per year, day and night, to 75,000 homes. In September, 2011, SolarReserve received a $737 million loan guarantee from the U.S. Department of Energy (DOE) for the project and broke ground. The project has a 25-year agreement with NV Energy for 100 percent of the electricity. Construction is complete and 

# 3. Using T5 and BERT for reranking

In [22]:
# reranker =  MonoT5()
reranker =  MonoBERT()

with open('bert.run', 'w') as runfile:

    for full_topic in tqdm(topic_docs):
        topic_id, facet_id = full_topic.split('/')
        topic = test_topics[topic_id]['title']
        facet = test_topics[topic_id]['facets'][facet_id]

        query = Query(topic + ' ' + facet + ' ' + facet)    
        texts = []
        for docid in topic_docs[full_topic]:
            texts.append(Text(paragraph_dictionary[docid], {'docid':docid}, 0))
        
        reranked = reranker.rerank(query, texts)

        doc_ids = []
        doc_scores = []
        for i in range(len(reranked)):
            doc_ids.append(reranked[i].metadata['docid'])
            doc_scores.append(reranked[i].score)
        
        sorted_tuples = sorted(zip(doc_scores, doc_ids))[::-1]
        doc_scores = [x for x,_ in sorted_tuples][0:20]
        doc_ids =[x for _,x in sorted_tuples][0:20]

        for i in range(20):
            runfile.write(full_topic + '\t' + 'Q0' + '\t' + doc_ids[i] + '\t'
                + str(i+1) + '\t' + str(doc_scores[i]) + '\t' + '$f' + '\n')



100%|██████████| 726/726 [5:15:41<00:00, 26.09s/it]


### Ranking Evaluation

In [23]:
import os
import subprocess
import sys
import platform

from pyserini.search import get_qrels_file
from pyserini.util import download_evaluation_script

script_path = download_evaluation_script('trec_eval')

!java -jar {script_path} -m map -m Rprec -m ndcg_cut.20 -m success.1 -m success.5 -m success.10  -c trec-car-benchmarkY3test-section.qrels bert.run

jtreceval-0.0.5-jar-with-dependencies.jar: 0.00B [00:00, ?B/s]

jtreceval-0.0.5-jar-with-dependencies.jar: 1.79MB [00:00, 3.50MB/s]                            


map                   	all	0.1679
Rprec                 	all	0.1945
ndcg_cut_20           	all	0.3052
success_1             	all	0.3234


# 4. Using summarization models for combining answers

### Preparing T5 model

In [ ]:
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

t5_model = T5ForConditionalGeneration.from_pretrained('t5-large')
t5_tokenizer = T5Tokenizer.from_pretrained('t5-large', )

# Testing T5:
text ="""
The US has "passed the peak" on new coronavirus cases, President Donald Trump said and predicted that some states would reopen this month.
The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, the highest for any country in the world.
At the daily White House coronavirus briefing on Wednesday, Trump said new guidelines to reopen the country would be announced on Thursday after he speaks to governors.
"We'll be the comeback kids, all of us," he said. "We want to get our country back."
The Trump administration has previously fixed May 1 as a possible date to reopen the world's largest economy, but the president said some states may be able to return to normalcy earlier than that.
"""

preprocess_text = text.strip().replace("\n","")
t5_prepared_Text = "summarize: "+preprocess_text
print("original text preprocessed: \n", preprocess_text)

tokenized_text = t5_tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)
t5_model = t5_model.to(device)

# summmarize 
summary_ids = t5_model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

output = t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nSummarized text: \n",output)


original text preprocessed: 
 The US has "passed the peak" on new coronavirus cases, President Donald Trump said and predicted that some states would reopen this month.The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, the highest for any country in the world.At the daily White House coronavirus briefing on Wednesday, Trump said new guidelines to reopen the country would be announced on Thursday after he speaks to governors."We'll be the comeback kids, all of us," he said. "We want to get our country back."The Trump administration has previously fixed May 1 as a possible date to reopen the world's largest economy, but the president said some states may be able to return to normalcy earlier than that.


Summarized text: 
 US has "passed the peak" on new coronavirus cases, president says. over 637,000 confirmed cases and over 30,826 deaths, the highest for any country in the world. new guidelines to reopen the country will be announced on Thursday, he says.


### Preparing pipeline summarizer

In [ ]:
bart_summarizer = pipeline('summarization')

# Test: 

text ="""
The US has "passed the peak" on new coronavirus cases, President Donald Trump said and predicted that some states would reopen this month.
The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, the highest for any country in the world.
At the daily White House coronavirus briefing on Wednesday, Trump said new guidelines to reopen the country would be announced on Thursday after he speaks to governors.
"We'll be the comeback kids, all of us," he said. "We want to get our country back."
The Trump administration has previously fixed May 1 as a possible date to reopen the world's largest economy, but the president said some states may be able to return to normalcy earlier than that.
"""

print('Original text: \n', text, '\n')

summary = bart_summarizer(text, max_length=1000,   min_length=30)
summary = summary[0]['summary_text']
print('Summarized text: \n', summary, '\n')

Your max_length is set to 1000, but you input_length is only 163. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


Original text: 
 
The US has "passed the peak" on new coronavirus cases, President Donald Trump said and predicted that some states would reopen this month.
The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, the highest for any country in the world.
At the daily White House coronavirus briefing on Wednesday, Trump said new guidelines to reopen the country would be announced on Thursday after he speaks to governors.
"We'll be the comeback kids, all of us," he said. "We want to get our country back."
The Trump administration has previously fixed May 1 as a possible date to reopen the world's largest economy, but the president said some states may be able to return to normalcy earlier than that.
 

Summarized text: 
  The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, the highest for any country in the world . New guidelines to reopen the country will be announced on Thursday after he speaks to governors . 



### Preparing BART

In [ ]:
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Test: 

text ="""
The US has "passed the peak" on new coronavirus cases, President Donald Trump said and predicted that some states would reopen this month.
The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, the highest for any country in the world.
At the daily White House coronavirus briefing on Wednesday, Trump said new guidelines to reopen the country would be announced on Thursday after he speaks to governors.
"We'll be the comeback kids, all of us," he said. "We want to get our country back."
The Trump administration has previously fixed May 1 as a possible date to reopen the world's largest economy, but the president said some states may be able to return to normalcy earlier than that.
"""

print('Original article:\n', text)

inputs = bart_tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)
summary_ids = bart_model.generate(inputs['input_ids'], num_beams=4, max_length=200, early_stopping=True)
summarized = [bart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]

print('Summarized article:\n', summarized)

Original article:
 
The US has "passed the peak" on new coronavirus cases, President Donald Trump said and predicted that some states would reopen this month.
The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, the highest for any country in the world.
At the daily White House coronavirus briefing on Wednesday, Trump said new guidelines to reopen the country would be announced on Thursday after he speaks to governors.
"We'll be the comeback kids, all of us," he said. "We want to get our country back."
The Trump administration has previously fixed May 1 as a possible date to reopen the world's largest economy, but the president said some states may be able to return to normalcy earlier than that.

Summarized article:
 The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, the highest for any country in the world. Trump said new guidelines to reopen the country would be announced on Thursday. "We'll be the comeback kids, all of us," he said.


### Applying summarization on all the topics

In [ ]:
def process_output(runfile_name, target_topic_id):

    retrieved_paragraphs = {}
    facet_articles = {}
    for facet in test_topics[target_topic_id]['facets']:
        retrieved_paragraphs[facet] = ['' for i in range(21)]
        facet_articles[facet] = ''

    with open('/content/run/' + runfile_name, 'r') as runfile:
        for line in runfile:
            elements = line.strip().split('\t')
            topic_id, facet_id = elements[0].split('/')

            if topic_id == target_topic_id:
                paragraph = paragraph_dictionary[elements[2]]
                rank = int(elements[3])
                retrieved_paragraphs[facet_id][rank] = paragraph

    # Creating initial, structured and facet articles:
    structured_article = test_topics[target_topic_id]['title'] + '\n\n'
    initial_article = test_topics[target_topic_id]['title'] + ' '
    for facet in test_topics[target_topic_id]['facets']:
        structured_article += test_topics[target_topic_id]['facets'][facet] + '\n'
        structured_article += retrieved_paragraphs[facet][1] + '\n'
        initial_article += retrieved_paragraphs[facet][1] + ' '
        
        for i in range(1, 21):
            facet_articles[facet] += retrieved_paragraphs[facet][i] + '\n'

    print('Initial article: \n', initial_article, '\n\n')
    print('structured article: \n', structured_article, '\n\n')

    # T5 summarization: 
    t5_prepared_Text = "summarize: " + initial_article
    tokenized_text = t5_tokenizer.encode(t5_prepared_Text, max_length=4096, return_tensors="pt").to(device)
    token_num = len(tokenized_text.tolist()[0])
    # summary_ids = t5_model.generate(tokenized_text,
    #                                 num_beams=4,
    #                                 no_repeat_ngram_size=2,
    #                                 # min_length= int(0.50 * token_num),
    #                                 min_length = 50,
    #                                 max_length= int(0.80 * token_num),
    #                                 # num_beam_groups = 20,
    #                                 early_stopping=True)
    # summarized_article = t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    # print('Summarized article by T5: \n', summarized_article, '\n')


    # Pipeline summarization:
    # summary = bart_summarizer(structured_article, max_length=int(0.80 * token_num), min_length=min(200, int(0.50 * token_num)))
    # summary = summary[0]['summary_text']
    # print('Summarized article by pipeline: \n', summary, '\n')


    # BART summarization
    inputs = bart_tokenizer([structured_article], return_tensors='pt', max_length=1024, truncation=True)
    summary_ids = bart_model.generate(inputs['input_ids'], num_beams=4, min_length=min(200, int(0.50 * token_num)), max_length=int(0.80 * token_num), early_stopping=True)
    summarized = [bart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
    print('Summarized article by BART: \n', summarized, '\n')
    

    # Creating the big article
    big_article = test_topics[target_topic_id]['title'] + '\n\n'
    for facet in facet_articles:
        big_article += test_topics[target_topic_id]['facets'][facet] + '\n'
        # facet_summary = bart_summarizer(facet_articles[facet][0:10000], max_length=150, min_length=50)
        # big_article += facet_summary[0]['summary_text'] + '\n'
        inputs = bart_tokenizer([facet_articles[facet]], return_tensors='pt',  max_length=1024, truncation=True)
        summary_ids = bart_model.generate(inputs['input_ids'], num_beams=4, min_length=50, max_length=150, early_stopping=True)
        big_article += [bart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0] + '\n'
 
    print('Big article:\n', big_article)

    with open('/content/results/' + runfile_name[0:-4] + '-' + target_topic_id + '.txt', 'w') as file:
        file.write('Structured article:\n\n' + structured_article + '\n\nShort Summary:\n\n' + summarized + '\n\nBig Summary:\n\n' + big_article)

    with open('/content/drive/MyDrive/IR-project/results/' + runfile_name[0:-4] + '-' + target_topic_id + '.txt', 'w') as file:
        file.write('Structured article:\n\n' + structured_article + '\n\nShort Summary:\n\n' + summarized + '\n\nBig Summary:\n\n' + big_article)


!mkdir results
for topic in tqdm(test_topics):
    process_output('dangnt-nlp.run', topic)

# process_output('dangnt-nlp.run', 'tqa2:L_0053')
# process_output('dangnt-nlp.run', 'tqa2:L_0264')

# process_output('ReRnak3_BERT.run', 'tqa2:L_0053')
# process_output('ReRnak2_BERT.run', 'tqa2:L_0053')
# process_output('IRIT3.run', 'tqa2:L_0053')

mkdir: cannot create directory ‘results’: File exists











  0%|          | 0/132 [00:00<?, ?it/s]

Initial article: 
 Earth Science and its branches Geology is the science and study of the solid and liquid matter that constitutes the Earth. The field of geology encompasses the study of the composition, structure, physical properties, dynamics, and history of Earth materials, and the processes by which they are formed, moved, and changed. The field is a major academic discipline, and is also important for mineral and hydrocarbon extraction, knowledge about and mitigation of natural hazards, some Geotechnical engineering fields, and understanding past climates and environments. Oceanography (compound of the Greek words ὠκεανός meaning "ocean" and γράφω meaning "write"), also known as oceanology, is the branch of Geography that studies the ocean. It covers a wide range of topics, including ecosystem dynamics; ocean currents, waves, and geophysical fluid dynamics; plate tectonics and the geology of the sea floor; and fluxes of various chemical substances and physical properties within t










  1%|          | 1/132 [01:34<3:27:05, 94.85s/it]

Big article:
 Earth Science and its branches

Geology
Geology is the study of the composition, structure, physical properties, dynamics, and history of Earth materials. Geologists typically study rock, sediment, soil, rivers, and natural resources. It is important for mineral and hydrocarbon extraction, knowledge about and mitigation of natural hazards, some Geotechnical engineering fields, and understanding past climates and environments.
Oceanography
Oceanography is a branch of Geography that studies the ocean. Paleoceanography studies the history of the oceans in the geologic past. The study of oceanography is divided into four branches. Oceanography is published quarterly by the Oceanography Society. The journal was first published in 1988.
Climatology and Meteorology
In contrast to meteorology, which studies short term weather systems lasting up to a few weeks, climatology studies the frequency and trends of those systems. Paleoclimatology seeks to reconstruct past climates by exa

...

Big article:
 mollusks and annelids

what are mollusks
Malacology has one of the largest collections of terrestrial and freshwater mollusks from the southeastern US. Gastropods comprise 83%, bivalves 16%, while all other mollusk classes combined <1% of the collection. The museum has the largest land snail collection in the world.
traits of mollusks
Not all mollusks are marine. There are numerous land and freshwater mollUSks, see for example snail and freshwater bivalves. Traits that increase the survival rate of an animal contribute to an animal's fitness, but selection will only favor such traits insofar as they improve the reproductive success of the organism.
mollusk reproduction
The reproductive system of gastropods varies greatly from one group to another within this very large and diverse taxonomic class of animals. Reproduction can be by sexual or asexual means. In some families of pulmonate land snails, one unusual feature of the reproductive system is the creation and utilizat










 63%|██████▎   | 83/132 [2:32:11<1:49:26, 134.01s/it]

Big article:
 introduction to vertebrates

what are vertebrates
Vertebrates comprise all species of animals within the subphylum Vertebrata (chordates with backbones) Vertebrates represent the overwhelming majority of the phylum Chordata, with currently about 64,000 species described. Vertebrate is closely related to the word vertebra, which refers to any of the bones or segments of the spinal column.
vertebral column
The vertebral column, also known as the backbone or spine, is part of the axial skeleton. It houses the spinal canal, a cavity that encloses and protects the spinal cord. The vertebrae have projections that allow for strong muscle attachment enabling locomotion without limbs.
vertebrate endoskeleton
An endoskeleton is an internal support structure of an animal, composed of mineralized tissue. Endoskeletons vary in complexity from functioning purely for support (as in the case of sponges), to serving as an attachment site for muscles and a mechanism for transmitting muscul










 64%|██████▎   | 84/132 [2:34:37<1:50:04, 137.60s/it]

Big article:
 fish

what are fish
Choo Thomas’ book stated: “What are all the fish for?” I asked. “This is food for the kingdom,” the Lord replied. The Lord simply pushed a button on the side of one oven and a fire began. He then assumed the role of a cook, right in front of me. He grilled the fish until both sides of it were brown. He ate the other half while I devoured the portion He gave me. It was delicious"
aquatic adaptations
Several animal groups have undergone aquatic adaptation, going from being purely terrestrial animals to living at least part of the time in water. Primarily or exclusively aquatic animals have re-evolved from terrestrial tetrapods multiple times. Examples include amphibians such as newts, reptiles such as crocodiles, sea turtles, ichthyosaurs, plesiosaurs and mosasaurs.
organ systems of fish
Not all fish fit comfortably in the above groups. Ocean sunfish, for example, have a completely different system. Many small fish use their pectoral fins for swimming as